In [10]:
import pandas as pd
import numpy as np
import torch    
import csv
import torch.nn as nn

In [12]:
modelV5_param=torch.load('../AbrahamsWork/Models/tttModelV5.pth')
modelV4_param=torch.load('../AbrahamsWork/Models/tttModelV4.pth')
modelHashV1_param=torch.load('../AbrahamsWork/Models/tttHashModelV1.pth')

In [3]:
data=pd.read_csv('../tic_tac_toe/tic_tac_toe.csv')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5478 entries, 0 to 5477
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   String      5478 non-null   object
 1   Value       5478 non-null   object
 2   Remoteness  5478 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 128.5+ KB


In [5]:
char_map = {"x":1, "o":2, "-":-1}
data['String'] = data['String'].apply(lambda s: [char_map[c] for c in s])

In [11]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.linear_relu = nn.Sequential(
            nn.Linear(9,16)
        )
        
    def forward(self, input):
        res = self.linear_relu(input)
        return res


In [24]:

# indices=['Lose0', 'Lose2', 'Lose4', 'Tie0', 'Tie1', 'Tie2', 'Tie3', 'Tie4', 'Tie5', 'Tie6', 'Tie7', 'Tie8', 'Tie9', 'Win1', 'Win3', 'Win5']

In [34]:
indices=['10','12','14','20','21','22','23','24','25','26','27','28','29','01','03','05']

In [35]:
indices[0][0],len(indices)

('1', 16)

In [30]:
data['Value'][0]

2

In [28]:
# Win,0
# Lose,1
# Tie,2
data=data.replace('Win',0)
data=data.replace('Lose',1)
data=data.replace('Tie',2)

In [44]:
modelV5=NeuralNetwork()
modelV5.load_state_dict(modelV5_param)
exception_table=[]
with torch.no_grad():
    for idx,i in enumerate(data['String'].values):
        inputs_string=torch.tensor(i,dtype=torch.float32)
        modelV5_output=modelV5(inputs_string)
        val_rem_idx=torch.argmax(modelV5_output)
        val,rem=int(indices[val_rem_idx][0]),int(indices[val_rem_idx][1])
        val_target,rem_target=data['Value'][idx],data['Remoteness'][idx]
        delta_val=val-val_target
        delta_rem=rem-rem_target
        if delta_val!=0 or delta_rem!=0:
            str_rep=''
            for j in i:
                if j==1:
                    str_rep+='x'
                elif j==2:
                    str_rep+='o'
                else:
                    str_rep+='-'
            exception_table.append([str_rep,delta_val,delta_rem])

        


In [43]:
len(exception_table)

2851

In [46]:
csv_file = "v5_exception_delta.csv"

import csv
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['String','Delta Value','Delta Remoteness'])
    for row in exception_table:
        writer.writerow(row)

In [47]:
excV5=pd.read_csv('v5_exception_delta.csv',header=0)
excV5

,String,Delta Value,Delta Remoteness
0,---------,0,-3
1,x--------,0,-2
2,-x-------,-2,-3
3,ox-------,-2,-2
4,-xo------,-2,-2
...,...,...,...
2846,o-o-oxxxx,-1,1
2847,-oo-oxxxx,-1,1
2848,o--ooxxxx,-1,1
2849,-o-ooxxxx,-1,1


In [48]:
last_column = excV5.iloc[:, -2:-1]
count = last_column.apply(lambda x: tuple(x), axis=1).value_counts()
print(count)

(-1,)    1403
(-2,)     890
(0,)      416
(1,)      119
(2,)       23
Name: count, dtype: int64


In [56]:

dict_exc=excV5.to_dict()
dict_exc.keys()
with open("0_delta_value_V5.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['String','Delta Remoteness'])
    for i in range(len(dict_exc['Delta Value'])):
        for key in dict_exc:
            row=[dict_exc[key][i] for key in dict_exc if key!="Delta Value"]
            row.append(0)
            # print(row)
            if key=="Delta Value" and dict_exc[key][i]==0:
                writer.writerow(row)


In [57]:
with open("1_delta_value_V5.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    for i in range(len(dict_exc['Delta Value'])):
        for key in dict_exc:
            row=[dict_exc[key][i] for key in dict_exc if key!="Delta Value"]
            # print(row)
            if key=="Delta Value" and dict_exc[key][i]==1:
                writer.writerow(row)
           

In [58]:
with open("2_delta_value_V5.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    for i in range(len(dict_exc['Delta Value'])):
        for key in dict_exc:
            row=[dict_exc[key][i] for key in dict_exc if key!="Delta Value"]
            # print(row)
            if key=="Delta Value" and dict_exc[key][i]==2:
                writer.writerow(row)
        

In [59]:
with open("-1_delta_value_V5.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    for i in range(len(dict_exc['Delta Value'])):
        for key in dict_exc:
            row=[dict_exc[key][i] for key in dict_exc if key!="Delta Value"]
            # print(row)
            if key=="Delta Value" and dict_exc[key][i]==-1:
                writer.writerow(row)
        

In [60]:
with open("-2_delta_value_V5.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    for i in range(len(dict_exc['Delta Value'])):
        for key in dict_exc:
            row=[dict_exc[key][i] for key in dict_exc if key!="Delta Value"]
            # print(row)
            if key=="Delta Value" and dict_exc[key][i]==-2:
                writer.writerow(row)
        

all:49KB
0&1&2&-1&-2<7+2+0.291+19+12=40.291KB
model:<2KB

original csv:119KB
<!-- 小了一半 -->